# Install Pyspark

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 20.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=1cae79eeb23d47a1f4a83a2c8ddc29e6285e0eda8cec45ba4b526e6c568a381f
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


# Imports

In [2]:
import pyspark
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
import re
import json
import itertools
import ast
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd
from operator import add
import numpy as np
import nltk 
from textblob import TextBlob
from textblob import Word
from nltk.corpus import stopwords


from sklearn import metrics 

In [3]:
# Imports
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import VectorAssembler, Imputer , StandardScaler
from pyspark.ml.classification import NaiveBayes

In [4]:
# Imports
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [5]:
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator

In [6]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


# Download Data

In [ ]:
#!pip install --upgrade --no-cache-dir gdown

In [ ]:
#!gdown --id 1Ba72Gnrjre0KPqhWbQPbyYR3XBguosGf

In [ ]:
#!unzip 'imdb-review-dataset.zip' -d '/content/drive/MyDrive/BigData_FinalProject/Data'

In [ ]:
#!cp '/content/imdb-review-dataset.zip' '/content/drive/MyDrive/BigData_FinalProject'

In [ ]:
'''
! pip install kaggle
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/BigData_FinalProject/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json
'''

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#! kaggle datasets download ebiswas/imdb-review-dataset

100% 2.69G/2.69G [00:23<00:00, 132MB/s]
100% 2.69G/2.69G [00:23<00:00, 124MB/s]


In [ ]:
#!unzip imdb-review-dataset.zip

Archive:  imdb-review-dataset.zip
  inflating: part-01.json            
  inflating: part-02.json            
  inflating: part-03.json            
  inflating: part-04.json            
  inflating: part-05.json            
  inflating: part-06.json            
  inflating: sample.json             


In [ ]:
#!unzip 'imdb-review-dataset.zip' -d '/content/drive/MyDrive/BigData_FinalProject/Data'

In [ ]:
'''
f = open('/content/drive/MyDrive/BigData_FinalProject/Data/sample.json')
data = json.load(f)
'''

In [ ]:
'''
with open(f'new.json', 'a') as f:
    json.dump(data[0:50000], f)
'''

In [ ]:
#new_data = json.load(open('new.json'))

In [ ]:
#!cp '/content/new.json' '/content/drive/MyDrive/BigData_FinalProject'

### 1. Data Cleaning

# Create Spark Context and Session

In [7]:
# Creating spark context
sc = pyspark.SparkContext(appName="Final_Project")
# Create a spark session
spark = SparkSession(sc)

## Reading the data by Spark (Dataframes)

In [8]:
# Reading the data and combine all parts in a single RDD
path = '/content/drive/MyDrive/BigData_FinalProject/part1files/*.json'
data_rdd = spark.read.option("multiline","true").json(path).rdd

In [9]:
data_rdd.take(5)

[Row(helpful=['5', '12'], movie='蜘蛛人：離家日 (2019)', rating='5', review_date='1 October 2019', review_detail='It\'s been just a few months since we\'ve witnessed the final episode of the Marvel saga that went on for ten years. A convergence point of such grandeur might suggest that from now on the Marvel world would follow a different path, giving us something truly new.However, the story of Spider-Man: Far from Home feels like an inverted mirror allusion to what the film itself tries to do with the established universe: selling us a bunch of illusions we\'d be eager to believe in simply because we wanna be fooled.With the Infinity Saga, we came to believe that all that superhero sci-fi extravaganza happening on screen actually has a deeper meaning, a layer for every major focus group, something most of us would not be ashamed to openly like. With that saga over, however, we\'re left with a bunch of secondary guys previously introduced mostly as filler, a few major characters past their p

In [10]:
df = data_rdd.toDF()

In [11]:
df.show(5)

+-------+---------------------+------+--------------+--------------------+---------+--------------------+--------------+-----------+
|helpful|                movie|rating|   review_date|       review_detail|review_id|      review_summary|      reviewer|spoiler_tag|
+-------+---------------------+------+--------------+--------------------+---------+--------------------+--------------+-----------+
|[5, 12]|蜘蛛人：離家日 (2019)|     5|1 October 2019|It's been just a ...|rw5154787|The illusion fact...|     Semisonic|          0|
| [2, 3]| Star Trek: A Piec...|     3|1 October 2019|I can't say this ...|rw5154791|I've Got A Grudge...|Dan1863Sickles|          0|
| [8, 9]|    流星花園 (2018– )|     3|1 October 2019|This rating has n...|rw5154789|Confusingly Enter...|   Cherry-sama|          0|
| [1, 1]| The Whole Truth (...|     6|1 October 2019|A rather tawdry m...|rw5154792|Murder Among the ...|richardchatten|          0|
| [1, 6]|     Tall Girl (2019)|     4|1 October 2019|She's only six on...|rw5154

In [12]:
#Statistically summarizing about the data
df.describe().show()

+-------+-------------+-----------------+---------------+--------------------+---------+-------------------+------------------+------------------+
|summary|        movie|           rating|    review_date|       review_detail|review_id|     review_summary|          reviewer|       spoiler_tag|
+-------+-------------+-----------------+---------------+--------------------+---------+-------------------+------------------+------------------+
|  count|        50000|            45925|          50000|               50000|    50000|              50000|             50000|             50000|
|   mean|         null|6.745998911268372|           null|                null|     null| 289.39285714285717| 791508.3571428572|           0.21898|
| stddev|         null|2.969629967663795|           null|                null|     null|   717.327094646707|223427.99288760958|0.4135591616970949|
|    min|#Alive (2020)|                1|1 February 2016|!! Contains a spo...|rw1985329|    !!Great Movie!!|  008_Bob-

In [ ]:
# Check number of rows and columns 
row = df.count()
col = len(df.columns)

print(f'Dimension of the Dataframe is: {(row,col)}')
print(f'Number of Rows are: {row}')
print(f'Number of Columns are: {col}')

In [12]:
#Printing Schema
df.printSchema() 

root
 |-- helpful: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- movie: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- review_date: string (nullable = true)
 |-- review_detail: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- review_summary: string (nullable = true)
 |-- reviewer: string (nullable = true)
 |-- spoiler_tag: long (nullable = true)



In [13]:
df.columns

['helpful',
 'movie',
 'rating',
 'review_date',
 'review_detail',
 'review_id',
 'review_summary',
 'reviewer',
 'spoiler_tag']

In [12]:
from pyspark.sql.types import StringType, DateType, FloatType

df2 = df \
  .withColumn("rating" ,
              df["rating"]
              .cast(FloatType()))   \
  .withColumn("spoiler_tag"  ,
              df["spoiler_tag"]
              .cast(FloatType()))

In [13]:
df2.printSchema() 

root
 |-- helpful: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- movie: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- review_date: string (nullable = true)
 |-- review_detail: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- review_summary: string (nullable = true)
 |-- reviewer: string (nullable = true)
 |-- spoiler_tag: float (nullable = true)



In [17]:
df2.describe().show()

+-------+-------------+-----------------+---------------+--------------------+---------+-------------------+------------------+------------------+
|summary|        movie|           rating|    review_date|       review_detail|review_id|     review_summary|          reviewer|       spoiler_tag|
+-------+-------------+-----------------+---------------+--------------------+---------+-------------------+------------------+------------------+
|  count|        50000|            45925|          50000|               50000|    50000|              50000|             50000|             50000|
|   mean|         null|6.745998911268372|           null|                null|     null| 289.39285714285717| 791508.3571428572|           0.21898|
| stddev|         null|2.969629967663795|           null|                null|     null|   717.327094646707|223427.99288760958|0.4135591616970949|
|    min|#Alive (2020)|              1.0|1 February 2016|!! Contains a spo...|rw1985329|    !!Great Movie!!|  008_Bob-

### Drop Missing Values

In [14]:
df2.show() #Displaying samples 

+--------+-------------------------+------+--------------+--------------------+---------+--------------------+--------------+-----------+
| helpful|                    movie|rating|   review_date|       review_detail|review_id|      review_summary|      reviewer|spoiler_tag|
+--------+-------------------------+------+--------------+--------------------+---------+--------------------+--------------+-----------+
| [5, 12]|    蜘蛛人：離家日 (2019)|   5.0|1 October 2019|It's been just a ...|rw5154787|The illusion fact...|     Semisonic|        0.0|
|  [2, 3]|     Star Trek: A Piec...|   3.0|1 October 2019|I can't say this ...|rw5154791|I've Got A Grudge...|Dan1863Sickles|        0.0|
|  [8, 9]|        流星花園 (2018– )|   3.0|1 October 2019|This rating has n...|rw5154789|Confusingly Enter...|   Cherry-sama|        0.0|
|  [1, 1]|     The Whole Truth (...|   6.0|1 October 2019|A rather tawdry m...|rw5154792|Murder Among the ...|richardchatten|        0.0|
|  [1, 6]|         Tall Girl (2019)|   4.0|1 

In [14]:
#Removing Rows with missing values
#Dataframe.na.drop removes any row with a NULL value

df3 = df2.dropna()

In [15]:
df3.show() #Displaying samples 

+--------+-------------------------+------+--------------+--------------------+---------+--------------------+--------------+-----------+
| helpful|                    movie|rating|   review_date|       review_detail|review_id|      review_summary|      reviewer|spoiler_tag|
+--------+-------------------------+------+--------------+--------------------+---------+--------------------+--------------+-----------+
| [5, 12]|    蜘蛛人：離家日 (2019)|   5.0|1 October 2019|It's been just a ...|rw5154787|The illusion fact...|     Semisonic|        0.0|
|  [2, 3]|     Star Trek: A Piec...|   3.0|1 October 2019|I can't say this ...|rw5154791|I've Got A Grudge...|Dan1863Sickles|        0.0|
|  [8, 9]|        流星花園 (2018– )|   3.0|1 October 2019|This rating has n...|rw5154789|Confusingly Enter...|   Cherry-sama|        0.0|
|  [1, 1]|     The Whole Truth (...|   6.0|1 October 2019|A rather tawdry m...|rw5154792|Murder Among the ...|richardchatten|        0.0|
|  [1, 6]|         Tall Girl (2019)|   4.0|1 

In [21]:
df3.describe().show()

+-------+-------------+-----------------+---------------+--------------------+---------+-------------------+------------------+-------------------+
|summary|        movie|           rating|    review_date|       review_detail|review_id|     review_summary|          reviewer|        spoiler_tag|
+-------+-------------+-----------------+---------------+--------------------+---------+-------------------+------------------+-------------------+
|  count|        45925|            45925|          45925|               45925|    45925|              45925|             45925|              45925|
|   mean|         null|6.745998911268372|           null|                null|     null| 289.39285714285717| 791508.3571428572|0.21445835601524224|
| stddev|         null|2.969629967663795|           null|                null|     null|   717.327094646707|223427.99288760958| 0.4104505304113301|
|    min|#Alive (2020)|              1.0|1 February 2016|!! Contains a spo...|rw1985329|    !!Great Movie!!|  00

In [22]:
# Check number of rows and columns 
row = df3.count()
col = len(df3.columns)

print(f'Dimension of the Dataframe is: {(row,col)}')
print(f'Number of Rows are: {row}')
print(f'Number of Columns are: {col}')

Dimension of the Dataframe is: (45925, 9)
Number of Rows are: 45925
Number of Columns are: 9


# Pipeline

In [ ]:
# Tokenize DataFrame
tokened = Tokenizer(inputCol="review_detail", outputCol="words")
tokened_transformed = tokened.transform(df)
tokened_transformed.show()

row = tokened_transformed.count()
col = len(tokened_transformed.columns)

print(f'Dimension of the Dataframe is: {(row,col)}')
print(f'Number of Rows are: {row}')
print(f'Number of Columns are: {col}')

+---------+--------------------+------+------------+--------------------+---------+--------------------+-------------------+-----------+--------------------+
|  helpful|               movie|rating| review_date|       review_detail|review_id|      review_summary|           reviewer|spoiler_tag|               words|
+---------+--------------------+------+------------+--------------------+---------+--------------------+-------------------+-----------+--------------------+
|   [0, 1]|Kill Bill: Vol. 2...|     8|24 July 2005|After seeing Tara...|rw1133942|Good follow up th...|OriginalMovieBuff21|          0|[after, seeing, t...|
| [11, 11]|Journey to the Un...|  null|24 July 2005|I have the entire...|rw1133943|    Excellent series|           sentra14|          0|[i, have, the, en...|
|   [2, 5]|   The Island (2005)|     9|24 July 2005|Once again the cr...|rw1133946|Not just about ac...|  GreenwheelFan2002|          0|[once, again, the...|
|   [2, 3]|Win a Date with T...|     3|24 July 2005|

In [ ]:
# Remove stop words
remover = StopWordsRemover(inputCol="words", outputCol="Wordsfiltered")
removed_frame = remover.transform(tokened_transformed)
removed_frame.show()

row = removed_frame.count()
col = len(removed_frame.columns)

print(f'Dimension of the Dataframe is: {(row,col)}')
print(f'Number of Rows are: {row}')
print(f'Number of Columns are: {col}')

+---------+--------------------+------+------------+--------------------+---------+--------------------+-------------------+-----------+--------------------+--------------------+
|  helpful|               movie|rating| review_date|       review_detail|review_id|      review_summary|           reviewer|spoiler_tag|               words|       Wordsfiltered|
+---------+--------------------+------+------------+--------------------+---------+--------------------+-------------------+-----------+--------------------+--------------------+
|   [0, 1]|Kill Bill: Vol. 2...|     8|24 July 2005|After seeing Tara...|rw1133942|Good follow up th...|OriginalMovieBuff21|          0|[after, seeing, t...|[seeing, tarantin...|
| [11, 11]|Journey to the Un...|  null|24 July 2005|I have the entire...|rw1133943|    Excellent series|           sentra14|          0|[i, have, the, en...|[entire, series, ...|
|   [2, 5]|   The Island (2005)|     9|24 July 2005|Once again the cr...|rw1133946|Not just about ac...| 

In [ ]:
# Run the hashing term frequency
hashing = HashingTF(inputCol="Wordsfiltered", outputCol="hashedValues")

# Transform into a DF
hashed_df = hashing.transform(removed_frame)
hashed_df.show()

row = hashed_df.count()
col = len(hashed_df.columns)

print(f'Dimension of the Dataframe is: {(row,col)}')
print(f'Number of Rows are: {row}')
print(f'Number of Columns are: {col}')

+---------+--------------------+------+------------+--------------------+---------+--------------------+-------------------+-----------+--------------------+--------------------+--------------------+
|  helpful|               movie|rating| review_date|       review_detail|review_id|      review_summary|           reviewer|spoiler_tag|               words|       Wordsfiltered|        hashedValues|
+---------+--------------------+------+------------+--------------------+---------+--------------------+-------------------+-----------+--------------------+--------------------+--------------------+
|   [0, 1]|Kill Bill: Vol. 2...|     8|24 July 2005|After seeing Tara...|rw1133942|Good follow up th...|OriginalMovieBuff21|          0|[after, seeing, t...|[seeing, tarantin...|(262144,[7625,842...|
| [11, 11]|Journey to the Un...|  null|24 July 2005|I have the entire...|rw1133943|    Excellent series|           sentra14|          0|[i, have, the, en...|[entire, series, ...|(262144,[3928,553...|


In [ ]:
df.head(1)

[Row(helpful=['0', '1'], movie='Kill Bill: Vol. 2 (2004)', rating='8', review_date='24 July 2005', review_detail="After seeing Tarantino's Kill Bill Vol: 1, I got to watch Vol. 2 the next day after seeing it. I liked the first one but didn't really know everything that was going on, but just knowing that she set off the kill 5 individuals who left her dead at her wedding. So I saw Kill Bill: Vol 2 and liked it highly. The movie answered all of my questions to the previous one and had much of a better story and was not unrealistic that much. We finally get to see Bill, who is played by David Carradine who had a really good role in the movie. There is a great conclusion to the movie and had a very good story, along with likable characters, my favorite being Budd. Overall, good movie that answers tons of questions. I recommend Kill Bill Vol. 2.Hedeen's Outlook: 8/10 *** B", review_id='rw1133942', review_summary='Good follow up that answers all the questions', reviewer='OriginalMovieBuff21

In [ ]:
hashed_df.head(1)

[Row(helpful=['0', '1'], movie='Kill Bill: Vol. 2 (2004)', rating='8', review_date='24 July 2005', review_detail="After seeing Tarantino's Kill Bill Vol: 1, I got to watch Vol. 2 the next day after seeing it. I liked the first one but didn't really know everything that was going on, but just knowing that she set off the kill 5 individuals who left her dead at her wedding. So I saw Kill Bill: Vol 2 and liked it highly. The movie answered all of my questions to the previous one and had much of a better story and was not unrealistic that much. We finally get to see Bill, who is played by David Carradine who had a really good role in the movie. There is a great conclusion to the movie and had a very good story, along with likable characters, my favorite being Budd. Overall, good movie that answers tons of questions. I recommend Kill Bill Vol. 2.Hedeen's Outlook: 8/10 *** B", review_id='rw1133942', review_summary='Good follow up that answers all the questions', reviewer='OriginalMovieBuff21

# Training

In [17]:
df3.printSchema() 

root
 |-- helpful: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- movie: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- review_date: string (nullable = true)
 |-- review_detail: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- review_summary: string (nullable = true)
 |-- reviewer: string (nullable = true)
 |-- spoiler_tag: float (nullable = true)



In [18]:
df_1 = df3['movie', 'spoiler_tag', 'review_detail', 'rating']

In [19]:
df_1.show(5)

+---------------------+-----------+--------------------+------+
|                movie|spoiler_tag|       review_detail|rating|
+---------------------+-----------+--------------------+------+
|蜘蛛人：離家日 (2019)|        0.0|It's been just a ...|   5.0|
| Star Trek: A Piec...|        0.0|I can't say this ...|   3.0|
|    流星花園 (2018– )|        0.0|This rating has n...|   3.0|
| The Whole Truth (...|        0.0|A rather tawdry m...|   6.0|
|     Tall Girl (2019)|        0.0|She's only six on...|   4.0|
+---------------------+-----------+--------------------+------+
only showing top 5 rows



In [20]:
# Train test split
training, testing = df_1.randomSplit([0.7, 0.3],1)

In [22]:
print("Training Dataset Count: " + str(df_1.count()))
print("Training Dataset Count: " + str(training.count()))
print("Test Dataset Count: " + str(testing.count()))

Training Dataset Count: 45925
Training Dataset Count: 32159
Test Dataset Count: 13766


In [21]:
# Show training data 
training.show()

+--------------------+-----------+--------------------+------+
|               movie|spoiler_tag|       review_detail|rating|
+--------------------+-----------+--------------------+------+
|       #Alive (2020)|        0.0|#Alive is a film ...|   7.0|
|       #Alive (2020)|        0.0|Haven't watched a...|   7.0|
|       #Alive (2020)|        1.0|#Alive was better...|   6.0|
|       #Alive (2020)|        1.0|Pretty enjoyable....|   8.0|
|#AnneFrank - Para...|        0.0|Let's face it, He...|   7.0|
|#AnneFrank - Para...|        0.0|Overall it's a wo...|   7.0|
|#AnneFrank - Para...|        0.0|The girl posting ...|   1.0|
|#AnneFrank - Para...|        0.0|This was one of t...|   4.0|
|   #BlackAF (2020– )|        0.0|Both my best frie...|  10.0|
|   #BlackAF (2020– )|        0.0|Brilliant show, a...|  10.0|
|   #BlackAF (2020– )|        0.0|I'll start by exp...|   1.0|
|#FriendButMarried...|        0.0|The 1st one is ok...|   3.0|
|      #Horror (2015)|        0.0|An artful and pro...|

In [22]:
# Find the data types
training.dtypes


[('movie', 'string'),
 ('spoiler_tag', 'float'),
 ('review_detail', 'string'),
 ('rating', 'float')]

In [23]:
# Linear REGRESSION MODEL 
featureCols = ['label_spoiler_tag', 'label_movie', 'review_features']
# Create all the steps for the pipeline
label_indexer = StringIndexer(inputCol='movie',outputCol='label_movie', stringOrderType="frequencyDesc").setHandleInvalid("keep")
label_indexer1 = StringIndexer(inputCol='spoiler_tag',outputCol='label_spoiler_tag')
tokenizer = Tokenizer(inputCol="review_detail", outputCol="Wordsfiltered")
stopremove = StopWordsRemover(inputCol='Wordsfiltered',outputCol='hashedValues')
hashingTF = HashingTF(inputCol="hashedValues", outputCol='review_features')
# put features into a feature vector column
assembler = VectorAssembler(inputCols=featureCols, outputCol="features") 

lr = LinearRegression(featuresCol='features', labelCol="rating", predictionCol='pred_rating', 
                       maxIter=10, regParam=0.3, elasticNetParam=0.8, standardization=False)

# Define pipeline
pipeline = Pipeline(stages=[label_indexer, label_indexer1, tokenizer, stopremove, hashingTF, assembler, lr])

# Fit the pipeline to training reviews.
lrmodel = pipeline.fit(training)

# Tranform the model with the testing data
predictions_lr = lrmodel.transform(testing)


In [24]:
predictions_lr.show()

+--------------------+-----------+--------------------+------+-----------+-----------------+--------------------+--------------------+--------------------+--------------------+------------------+
|               movie|spoiler_tag|       review_detail|rating|label_movie|label_spoiler_tag|       Wordsfiltered|        hashedValues|     review_features|            features|       pred_rating|
+--------------------+-----------+--------------------+------+-----------+-----------------+--------------------+--------------------+--------------------+--------------------+------------------+
|#AnneFrank - Para...|        0.0|But this panderin...|   4.0|     4126.0|              0.0|[but, this, pande...|[pandering, youth...|(262144,[8804,211...|(262146,[1,8806,2...| 6.813491207672259|
|#AnneFrank - Para...|        0.0|Is this a docu-dr...|   5.0|     4126.0|              0.0|[is, this, a, doc...|[docu-drama, exer...|(262144,[3121,941...|(262146,[1,3123,9...| 6.813491207672259|
|   #BlackAF (2020– 

In [25]:
# Evaluate Logistic Regression model
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="pred_rating", \
                 labelCol="rating",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(predictions_lr))


R Squared (R2) on test data = 0.00322494


In [26]:
# Evaluate Logistic Regression model
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="pred_rating", \
                 labelCol="rating",metricName="rmse")
print("RMSE) on test data = %g" % lr_evaluator.evaluate(predictions_lr))


RMSE) on test data = 3.02869


In [16]:
df3.printSchema() 

root
 |-- helpful: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- movie: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- review_date: string (nullable = true)
 |-- review_detail: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- review_summary: string (nullable = true)
 |-- reviewer: string (nullable = true)
 |-- spoiler_tag: float (nullable = true)



In [17]:
df_2 = df3['review_detail', 'rating']

In [18]:
df_2.show(5)

+--------------------+------+
|       review_detail|rating|
+--------------------+------+
|It's been just a ...|   5.0|
|I can't say this ...|   3.0|
|This rating has n...|   3.0|
|A rather tawdry m...|   6.0|
|She's only six on...|   4.0|
+--------------------+------+
only showing top 5 rows



In [19]:
# Train test split
training1, testing1 = df_2.randomSplit([0.7, 0.3],1)

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator


# GBT REGRESSION MODEL 
featureCols = ['review_features']
# Create all the steps for the pipeline
tokenizer = Tokenizer(inputCol="review_detail", outputCol="Wordsfiltered")
stopremove = StopWordsRemover(inputCol='Wordsfiltered',outputCol='hashedValues')
hashingTF = HashingTF(inputCol="hashedValues", outputCol='review_features')
# put features into a feature vector column
assembler = VectorAssembler(inputCols=featureCols, outputCol="features") 

# Train a GBT model.
gbt = GBTRegressor(featuresCol="features", labelCol="rating", maxIter=10)



# Define pipeline
pipeline = Pipeline(stages=[tokenizer, stopremove, hashingTF, assembler, gbt])

# Fit the pipeline to training reviews.
gbtmodel = pipeline.fit(training1)

# Tranform the model with the testing data
predictions = gbtmodel.transform(testing1)



# Select example rows to display.
predictions.select("prediction", "rating", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="rating", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

gbtModel = gbtmodel.stages[6]
print(gbtModel)  # summary only


In [25]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Linear REGRESSION MODEL 
featureCols = ['review_features']
# Create all the steps for the pipeline
tokenizer = Tokenizer(inputCol="review_detail", outputCol="Wordsfiltered")
stopremove = StopWordsRemover(inputCol='Wordsfiltered',outputCol='hashedValues')
hashingTF = HashingTF(inputCol="hashedValues", outputCol='review_features')
# put features into a feature vector column
#assembler = VectorAssembler(inputCols="review_features", outputCol="features") 

# Train a DT model.
gbt = DecisionTreeRegressor(featuresCol="review_features", labelCol="rating")
# Define pipeline
pipeline = Pipeline(stages=[tokenizer, stopremove, hashingTF, gbt])
# Fit the pipeline to training reviews.
gbtmodel = pipeline.fit(training1)
# Tranform the model with the testing data
predictions = gbtmodel.transform(testing1)




In [ ]:
# Select example rows to display.
predictions.show(5)
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="rating", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

gbtModel = gbtmodel.stages[3]
print(gbtModel)  # summary only